## Import Library and path

In [99]:
import pandas as pd
import json
DATA_FILE_PATH = "test_data/data.json"

## Load Data

In [100]:
try:
    records = []
    with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f):
            try:
                records.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line {line_number + 1}: {e}")

    df = pd.DataFrame(records)
    print(f"Successfully loaded {len(df)} records.")

except FileNotFoundError:
    print(f"Error: Data file not found at {DATA_FILE_PATH}")
except Exception as e:
    print(f"An error occurred during data exploration: {e}")

Successfully loaded 46279 records.


## Initial Exploration and Understanding

### View basic info

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46279 entries, 0 to 46278
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   sacc_items                46279 non-null  int64  
 1   work_orders               46279 non-null  int64  
 2   female_items              46279 non-null  int64  
 3   is_newsletter_subscriber  46279 non-null  object 
 4   male_items                46279 non-null  int64  
 5   afterpay_payments         46279 non-null  int64  
 6   msite_orders              46279 non-null  int64  
 7   wftw_items                46279 non-null  int64  
 8   mapp_items                46279 non-null  int64  
 9   orders                    46279 non-null  int64  
 10  cc_payments               46279 non-null  int64  
 11  curvy_items               46279 non-null  int64  
 12  paypal_payments           46279 non-null  int64  
 13  macc_items                46279 non-null  int64  
 14  cancel

**OBSERVATION**:
- Dataset contain `46279` rows and `43` columns
- Column `coupon_discount_applied` have 46279 - 36074 = `10205` null value

**ACTION**:
- Fill null values in `coupon_discount_applied` column with 0 since missing values likely indicate no discount was applied

### Get descriptive statistics:

In [102]:
COLUMN_TYPES = {
    'integer': df.select_dtypes(include=['int64', 'Int64']).columns.tolist(),
    'float': df.select_dtypes(include=['float64']).columns.tolist(),
    'string': df.select_dtypes(include=['object', 'string']).columns.tolist()
}

integer_columns = COLUMN_TYPES['integer']
float_columns = COLUMN_TYPES['float']
string_columns = COLUMN_TYPES['string']

In [103]:
print("--- Descriptive Statistics for Integer Columns ---")
pd.set_option('display.max_columns', None)
print(df[integer_columns].agg(['count', 'mean', 'min', 'max']).round(2))
pd.reset_option('display.max_columns')

print("--- Descriptive Statistics for Float Columns ---")
print(df[float_columns].agg(['count', 'mean', 'min', 'max']).round(2))

print("--- Descriptive Statistics for String Columns ---")
print(df[string_columns].agg(['count', 'nunique']).round(2))


--- Descriptive Statistics for Integer Columns ---
       sacc_items  work_orders  female_items  male_items  afterpay_payments  \
count    46279.00     46279.00      46279.00    46279.00           46279.00   
mean         0.09         0.24          6.47        1.72               0.05   
min          0.00         0.00          0.00        0.00               0.00   
max         29.00        84.00        537.00      273.00               1.00   

       msite_orders  wftw_items  mapp_items    orders  cc_payments  \
count      46279.00    46279.00    46279.00  46279.00     46279.00   
mean           0.98        1.63        0.93      4.11         0.64   
min            0.00        0.00        0.00      1.00         0.00   
max          172.00      261.00      151.00    665.00         1.00   

       curvy_items  paypal_payments  macc_items   cancels   returns  \
count     46279.00         46279.00    46279.00  46279.00  46279.00   
mean          0.04             0.49        0.57      0.05   

**OBSERVATION**:

**Integer Columns Analysis:**

1.  **`days_since_last_order`**:
    *   The `mean` value is approximately `24592.68` days (around 67.3 years) and the `max` is `51840` days (around 142 years). These values are implausibly high for "days since last order" and strongly suggest this column is corrupted or its unit is not days (e.g., it might be hours, minutes, or has an erroneous scaling factor).
    *   This contrasts sharply with `days_since_first_order` (mean ~3.7 years), which appears more reasonable.
    *   **This is a primary candidate for a corrupted column.**

2.  **Payment Columns & `different_addresses`**:
    *   Columns like `cc_payments`, `paypal_payments`, `afterpay_payments`, `apple_payments`, and `different_addresses` all have a `max` value of `1`.
    *   This suggests they might be acting as boolean flags (0 or 1 indicating presence/absence) rather than "Number of times..." as per some column descriptions. If they are indeed counts, a max of 1 for all customers is highly unlikely for active payment methods. This needs clarification.

3.  **Order Aggregation Columns**:
    *   `other_collection_orders` and `desktop_orders` have a `max` value of `665`, which is identical to the `max` value for the `orders` column. This implies that for at least one customer, *all* their orders (665 of them) were either "other collection" or "desktop" orders. While possible, this warrants a consistency check at the record level to ensure that the sum of order channels/destinations matches total `orders` for each customer.

4.  **General Counts**:
    *   `orders` and `items` both have a `min` of `1`. This indicates that every record in this dataset pertains to a customer who has made at least one order and purchased at least one item.
    *   Other item counts (e.g., `female_items`, `male_items`) and activity counts (e.g., `cancels`, `returns`) have plausible ranges, but their consistency with total `orders` and `items` should be verified

**Float Columns Analysis:**

1.  **`revenue`**:
    *   The `min` is `0.00`, which is plausible (e.g., all items returned, or initial order not yet fully processed/paid). The `max` is `354700.16`, showing a wide range of customer spending.
    *   The presence of zero revenue cases requires further investigation to understand if these represent returns, cancelled orders, or data quality issues

**String Columns Analysis:**

1.  **`customer_id`**:
    *   The `count` is `46279`, but `nunique` (number of unique values) is `46030`.
    *   This indicates there are **`249` duplicate `customer_id` values**. This is a critical data quality issue that needs to be addressed, as customer IDs should uniquely identify customers.

2.  **`is_newsletter_subscriber`**:
    *   `nunique` is `2`. This suggests the column has been successfully processed into two distinct categories.

**Summary & Next Steps:**

*   **Corrupted Columns Candidates:** `days_since_last_order` show strong signs of corruption or systemic error.
*   **Data Definition Mismatch:** Payment-related columns and `different_addresses` might be boolean flags rather than counts.
*   **Duplicate Customer IDs:** The presence of 249 duplicate `customer_id`s must be investigated and resolved.
*   **Order Channel Validation:** Need to verify that total `orders` equals the sum of all order channel counts (e.g., `desktop_orders` + `mobile_orders` + `app_orders`) for each customer.


## Clean Data & Handle corrupted/abnormal columns

### Handle missing values by fill with 0 and convert data types

In [104]:
def convert_column_type(df: pd.DataFrame, cols: list[str], dtype: str) -> None:
    """
    Convert column to specified data type, handling NaN values and printing warnings.
    
    Args:
        df: pandas DataFrame
        cols: list of column names to convert
        dtype: data type to convert to ('int', 'float', or 'str')
    """
    for col in cols:
        if col in df.columns:
            if dtype in ['int', 'float']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
                
                nan_count = df[col].isna().sum()
                if nan_count > 0:
                    print(f"Warning: Columzn '{col}' has {nan_count} NaN values.")
                
                df[col] = df[col].fillna(0)
                df[col] = df[col].astype(dtype)
            
            elif dtype == 'str':
                df[col] = df[col].astype(str)

convert_column_type(df, integer_columns, 'int') 
convert_column_type(df, float_columns, 'float')   
convert_column_type(df, string_columns, 'str')   

### Handle duplicate `customer_id`

In [105]:
duplicate_records = df[df.duplicated(subset=['customer_id'], keep=False)]
num_duplicates = len(duplicate_records)

print(f"Found {num_duplicates} duplicate customer_id records")

# Get counts of each duplicated customer_id
dup_counts = df['customer_id'].value_counts()
duplicate_customer_ids = dup_counts[dup_counts > 1]

print(f"Number of unique customer_ids with multiple records: {len(duplicate_customer_ids)}")

print('Look like each record duplicate two times')

Found 498 duplicate customer_id records
Number of unique customer_ids with multiple records: 249
Look like each record duplicate two times


**Check if all duplicates appear exactly twice**

In [106]:
is_all_duplicates_twice = all(count == 2 for count in duplicate_customer_ids)
if is_all_duplicates_twice:
    print('Each duplicated customer_id appears exactly twice in the dataset')
else:
    print('Duplicate patterns:')
    print(duplicate_customer_ids.value_counts().to_frame('count'))

Each duplicated customer_id appears exactly twice in the dataset


**Check values are identical in all other columns**

In [107]:
columns = integer_columns + float_columns + string_columns
inconsistent_duplicates_count = 0

for customer_id, _ in duplicate_customer_ids.items():
    customer_records = duplicate_records[duplicate_records['customer_id'] == customer_id]
    
    # Check if values are identical across duplicates
    for col in columns:
        if not customer_records[col].nunique() == 1:
            inconsistent_duplicates_count += 1
    

if inconsistent_duplicates_count > 0:
    print(f"Found {inconsistent_duplicates_count} inconsistent values across duplicate records")
else:
    print("All values are consistent across duplicate records")


All values are consistent across duplicate records


Safely remove the redundant recors, keeping only one instance

In [108]:
df = df.drop_duplicates(subset=['customer_id'], keep='first')

### Fix cases where `days_since_last_order` exceeds `days_since_first_order`

In [109]:
if 'days_since_first_order' in df.columns and 'days_since_last_order' in df.columns:
    inconsistent_days = df[df['days_since_last_order'] > df['days_since_first_order']]
    if not inconsistent_days.empty:
        print(f"There are {len(inconsistent_days)} records where 'days_since_last_order' > 'days_since_first_order'.")

There are 43365 records where 'days_since_last_order' > 'days_since_first_order'.


In [110]:
date_difference = inconsistent_days['days_since_last_order'] - inconsistent_days['days_since_first_order']
print(f"Range of difference between dates:")
print(f"Minimum difference: {date_difference.min():.0f} days")
print(f"Maximum difference: {date_difference.max():.0f} days")
print(f"Mean difference: {date_difference.mean():.1f} days")


Range of difference between dates:
Minimum difference: 1 days
Maximum difference: 49680 days
Mean difference: 24638.7 days


**OBSERVATION**:
- The range of differences between `days_since_last_order` and `days_since_first_order` is quite large. 
- This suggests these values may have been accidentally swapped during data entry. 

**ACTION**:
- I will swap them back to maintain logical order where last_order >= first_order.

In [111]:
#Create a mask for records where days_since_last_order > days_since_first_order
swap_mask = df['days_since_last_order'] > df['days_since_first_order']

# Swap the values using a temporary column
df.loc[swap_mask, 'temp'] = df.loc[swap_mask, 'days_since_first_order']
df.loc[swap_mask, 'days_since_first_order'] = df.loc[swap_mask, 'days_since_last_order']
df.loc[swap_mask, 'days_since_last_order'] = df.loc[swap_mask, 'temp']

# Drop the temporary column
df.drop(columns=['temp'], inplace=True)

# Check again
inconsistent_days = df[df['days_since_last_order'] > df['days_since_first_order']]
if inconsistent_days.empty:
    print(f"There are {len(inconsistent_days)} records where 'days_since_last_order' > 'days_since_first_order'.")


There are 0 records where 'days_since_last_order' > 'days_since_first_order'.


### Check consistency of Item Categories

**Check consistency between items and gender categories**

In [47]:
item_categories = ['female_items', 'male_items', 'unisex_items']
if all(col in df.columns for col in item_categories + ['items']):
    df['sum_item_categories'] = df[item_categories].sum(axis=1, skipna=False)
    inconsistent_items_sum = df[df['items'] != df['sum_item_categories']]
    inconsistent_items_sum = inconsistent_items_sum[
        inconsistent_items_sum['items'].notna() & 
        inconsistent_items_sum['sum_item_categories'].notna()
    ]
    
    if not inconsistent_items_sum.empty:
        print(f"Found {len(inconsistent_items_sum)} records where 'items' != sum of gender categories")
        
        # Check if items is greater than sum of gender categories
        items_greater = inconsistent_items_sum[
            inconsistent_items_sum['items'] > inconsistent_items_sum['sum_item_categories']
        ]
        if not items_greater.empty:
            print(f"  - {len(items_greater)} records where 'items' > sum of gender categories")
            
        # Check if items is less than sum of gender categories  
        items_lesser = inconsistent_items_sum[
            inconsistent_items_sum['items'] < inconsistent_items_sum['sum_item_categories']
        ]
        if not items_lesser.empty:
            print(f"  - {len(items_lesser)} records where 'items' < sum of gender categories")
    else:
        print("No inconsistencies found between 'items' and sum of gender categories")
        
    df.drop(columns=['sum_item_categories'], inplace=True)


No inconsistencies found between 'items' and sum of gender categories


**Check consistency between items and detail categories**

In [17]:
def check_item_categories(df: pd.DataFrame, detailed_item_cols: list[str]) -> None:
        if all(col in df.columns for col in detailed_item_cols + ['items']):
                df['sum_detailed_items'] = df[detailed_item_cols].sum(axis=1, skipna=False)
                inconsistent_detailed_items_sum = df[df['items'] != df['sum_detailed_items']]
                inconsistent_detailed_items_sum = inconsistent_detailed_items_sum[inconsistent_detailed_items_sum['items'].notna() & inconsistent_detailed_items_sum['sum_detailed_items'].notna()]

                if not inconsistent_detailed_items_sum.empty:
                        print(f" There are {len(inconsistent_detailed_items_sum)} records where 'items' != sum of all detailed item category columns.")
                        
                        # Check if items is greater than sum_detailed_items
                        items_greater = inconsistent_detailed_items_sum[inconsistent_detailed_items_sum['items'] > inconsistent_detailed_items_sum['sum_detailed_items']]
                        if not items_greater.empty:
                                print(f"    - {len(items_greater)} records where 'items' > sum of detailed items")
                        
                        # Check if items is less than sum_detailed_items    
                        items_lesser = inconsistent_detailed_items_sum[inconsistent_detailed_items_sum['items'] < inconsistent_detailed_items_sum['sum_detailed_items']]
                        if not items_lesser.empty:
                                print(f"    - {len(items_lesser)} records where 'items' < sum of detailed items")
                else:
                        print(f"No records where 'items' != sum of all detailed item category columns.")
                df.drop(columns=['sum_detailed_items'], inplace=True)

detailed_item_cols = ['wapp_items', 'wftw_items', 'mapp_items', 'wacc_items', 'macc_items', 
                        'mftw_items', 'wspt_items', 'mspt_items', 'curvy_items', 'sacc_items']
check_item_categories(df, detailed_item_cols)

 There are 16970 records where 'items' != sum of all detailed item category columns.
    - 8240 records where 'items' > sum of detailed items
    - 8730 records where 'items' < sum of detailed items


**OBSERVATION**:
- Case `items` < `sum_detailed_items`:

    I assume that the `items` total is incorrectly calculated 
    

- Case `items` > `sum_detailed_items`:

    I assume that there are other categories of items not listed that contribute to the total
    
**ACTION**: 
I notice there are inconsistencies between `items` and sum of detailed item categories.
However, I cannot definitively resolve these discrepancies without additional business rules and validation criteria:
- When `items` < sum of detailed items: Need business rules to determine if `items` total is incorrect or if there's double-counting in detailed categories
- When `items` > sum of detailed items: Need to confirm if missing categories exist or if `items` total includes items not tracked in detailed breakdown


## Save data

In [112]:
df.to_csv('test_data/cleaned_data.csv', index=False)
print("Data saved successfully to 'cleaned_data.csv'")

Data saved successfully to 'cleaned_data.csv'
